In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians


In [2]:
world_map = folium.Map()
world_map

In [3]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

In [4]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [6]:
# Initial the map
# site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for _, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    circle = folium.Circle(
        coordinate, 
        radius=50, 
        color='blue', 
        fill=True
        ).add_child(folium.Popup(row['Launch Site']))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)


# Task 2: Mark the success/failed launches for each site on the map
1. ***Proximity to the Coastline:*** All launch sites are located near oceans. This is due to safety considerations, as spent rocket stages fall into the ocean, minimizing risks to people and infrastructure.

2. ***Latitude:*** Launch sites on the East Coast (Florida) are closer to the equator, which allows rockets to take advantage of Earth's rotation to increase their velocity, reducing fuel costs (especially important for launches to geostationary orbit).

3. ***Purpose of the Site:*** The VAFB SLC-4E site in California is used for launches to polar orbits, which require flight trajectories over the ocean rather than over densely populated areas. Additionally, California (VAFB) has generally stable weather conditions, making it a suitable region for launches.

In [7]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [8]:
marker_cluster = MarkerCluster()

In [9]:
# Определяем цвет маркеров
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Создаём кластер маркеров
marker_cluster = MarkerCluster()

# Добавляем маркеры в кластер
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=row['marker_color']),
        popup=f"Launch Site: {row['Launch Site']}<br>Class: {row['class']}"
    ).add_to(marker_cluster)

site_map.add_child(marker_cluster)

In [10]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)


In [11]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [12]:
# Новые координаты точки запуска
launch_site_lat, launch_site_lon = 28.5632, -80.57682

# Координаты ближайшей точки на побережье
coastline_lat, coastline_lon = 28.56254, -80.56787

# Расчёт расстояния между точкой запуска и побережьем
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Добавление маркера с крестиком для точки запуска
start_point_marker = folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(10, 10),  # Центрирование крестика
        html='<div style="font-size: 16px; color: red;"><b>✕</b></div>'  # Красный крестик
    )
)
site_map.add_child(start_point_marker)

# Добавление маркера для точки побережья
coastline_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color: blue;"><b>%s</b></div>' % 
             "{:10.2f} KM".format(distance_coastline)  # Отображение расстояния
    )
)
site_map.add_child(coastline_marker)

# Добавление линии между точкой запуска и побережьем
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='blue',
    weight=2,
    opacity=0.7
).add_to(site_map)

# Сохранение карты
site_map.save('spacex_launch_with_updated_coordinates.html')

# # Отображение карты
# site_map


In [13]:
# Координаты
city_lat, city_lon = 28.07774, -80.61527

# Расчёт расстояний от точки запуска до объектов
distance_to_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Линия к ближайшему городу
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
    color='blue',
    weight=2,
    opacity=0.7
).add_to(site_map)

# Метка для ближайшего города
city_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color: blue;"><b>City: {:.2f} KM</b></div>'.format(distance_to_city)
    )
)

site_map.add_child(city_marker)

In [14]:
railway_lat, railway_lon = 28.56343, -80.58688
distance_to_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

folium.PolyLine(
    locations=[[launch_site_lat,launch_site_lon], [railway_lat, railway_lon]],
    color='green',
    weight=2,
    opacity=0.7
).add_to(site_map)

railway_marker = folium.Marker(
    location=[railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="front-size: 12; color: green;"><b>RailWay:{:.2f} km</div>'.format(distance_to_railway)
    )
)

site_map.add_child(railway_marker)

In [15]:
highway_lat, highway_lon = 28.56289, -80.57073
distance_to_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

folium.PolyLine(
    locations=[[launch_site_lat,launch_site_lon], [highway_lat, highway_lon]],
    color='red',
    weight=2,
    opacity=0.7
).add_to(site_map)

highway_marker = folium.Marker(
    location=[highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="front-size: 12; color: red;"><b>highway:{:.2f} km</div>'.format(distance_to_highway)
    )
)

site_map.add_child(highway_marker)

In [16]:
# Save map
site_map.save('spacex_launch_map.html')

# Show map
site_map
